In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(4)

## Load Model

In [4]:
name = 'baseline_any_xse_resnext50_appian_fromscratch'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=1000, pretrained='imagenet')

In [7]:
# m.avg_pool = nn.AdaptiveAvgPool2d(1)
# m.last_linear = nn.Linear(in_features=2048, out_features=6)

m.avg_pool = AdaptiveConcatPool2d(1)
m.last_linear = nn.Linear(in_features=2048*2, out_features=6)

In [8]:

def appianish_cycle_train(learn, get_data, no_pretrain=False, schedule=None, lr=3e-4):
    do_slice=freeze= not no_pretrain
    if not schedule: schedule = [(64,192), (64,352), (64,512), (64,None)]

    """
    learn.loss_func = get_loss()
    bs,sz = schedule[3]
    learn.dls = get_data(bs, None, use_wgt=False)
    do_fit(learn, 1, lr*2/3*2/3, freeze=False, **no_1cycle)
    """

    if no_pretrain:
        learn.loss_func = get_loss()
        learn.dls = get_data(64, 160, use_wgt=False)
        do_fit(learn, 4, lr, freeze=False, **no_1cycle)

    #if not schedule: schedule = [(64,192), (64,352), (64,512), (64,None)]
    #if not schedule: schedule = [(16,192),(16,352),(16,512),(16,None)]
    learn.loss_func = get_loss()
    bs,sz = schedule[0]
    learn.dls = get_data(bs, 192, use_wgt=True)
    do_fit(learn, 2, lr, do_slice=do_slice, freeze=freeze , **no_1cycle)
    bs,sz = schedule[1]
    learn.dls = get_data(bs, 352, use_wgt=True)
    do_fit(learn, 2, lr*2/3, do_slice=do_slice, freeze=freeze, **no_1cycle)
    bs,sz = schedule[2]
    learn.dls = get_data(bs, 512, use_wgt=True)
    do_fit(learn, 1, lr*2/3*2/3, do_slice=False, freeze=False, **no_1cycle)

    learn.loss_func = get_loss()
    bs,sz = schedule[3]
    learn.dls = get_data(bs, None, use_wgt=False)
    do_fit(learn, 1, lr*2/3*2/3, freeze=False, **no_1cycle)
    do_fit(learn, 1, lr*2/3*2/3, freeze=False, div=1, pct_start=0.01)

In [9]:
dls = get_data(384, 192, splits=Meta.splits, img_dir=path_jpg256)
learn = get_learner(dls, m)

In [10]:
lr = 4e-3

## Training

In [11]:
do_fit(learn, 2, lr, **no_1cycle)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 

In [13]:
learn.dls = get_data(196, 352, splits=Meta.splits, img_dir=path_jpg)
do_fit(learn, 2, lr*1/3, freeze=False, **no_1cycle)
learn.save(f'runs/{name}-2')

RuntimeError: CUDA out of memory. Tried to allocate 484.00 MiB (GPU 4; 23.65 GiB total capacity; 22.16 GiB already allocated; 126.00 MiB free; 512.36 MiB cached)

In [ ]:
learn.dls = get_data(128, 512, splits=Meta.splits, img_dir=path_jpg)
do_fit(learn, 1, lr*1/9, freeze=False, **no_1cycle)
learn.save(f'runs/{name}-3')

In [ ]:
learn.dls = get_data(128, None, splits=Meta.splits, img_dir=path_jpg)
do_fit(learn, 1, lr*1/9, freeze=False, **no_1cycle)
learn.save(f'runs/{name}-4')

## Submission

In [ ]:
learn.load(f'runs/{name}-1')
sub_fn = f'subm/{name}'

In [ ]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')

In [ ]:
ob = learn.dls.one_batch()

In [ ]:
preds,targs = learn.get_preds()

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [16]:
api.competition_submit(f'{sub_fn}.csv', f'{name} appian se_resnext50', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.96MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [17]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13820147,
 'totalBytes': 27277209,
 'date': '2019-12-16T19:19:33.217Z',
 'description': 'baseline_any_xse_resnext_512_appian appian se_resnext50',
 'errorDescription': None,
 'fileName': 'baseline_any_xse_resnext_512_appian.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13820147/13820147.raw'}

## Save Predictions

In [11]:
class FeatureModel(nn.Module):
    def __init__(self, m): 
        super().__init__()
        self.m = m
        
    def forward(self, x):
        return self.m.avg_pool(self.m.features(x))

In [12]:
#export
class DummyLoss:
    def __call__(self, p, *t, **kwargs): return torch.tensor(0, device=p.device).float()

In [13]:
#export
def save_features(learn, feat_path):
    preds,targs = learn.get_preds(dl=learn.dls.valid)
    val_ids = dls.valid.dataset.items
    feat_path.mkdir(exist_ok=True)
    for idx,pred in progress_bar(zip(val_ids, preds), total=len(val_ids)):
        fn = feat_path/f'{idx}'
        if fn.exists(): continue
        np.save(str(fn), pred.squeeze().numpy())

In [14]:
#export
path_feat512 = path/'features_512'
path_feat512_tst = path/'tst_features_512'

In [15]:
len(dls.valid)

1306

In [16]:
dls = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg', sl=slice(50000, 200000))
# dls = get_data(48, 512, splits=Meta.splits_sample, img_dir=path_jpg)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(f'runs/{name}-1')
learn.model = FeatureModel(learn.model)

In [17]:
save_features(learn, path_feat512)

In [18]:
import gc
gc.collect()

20

In [ ]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')
save_features(learn, path_feat512_tst)